In [1]:
import os
from dfply import *
import pandas as pd
import datetime
np.set_printoptions(suppress=True) 
os.chdir(r"D:\03lecture\QT\codeBackup\data")
from math import *

In [2]:
def end_of_month(any_day):
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)

In [3]:
# formatting of msf
msf = pd.read_csv("msf.csv")
msf["date"] = pd.to_datetime(msf["date"], format='%Y%m%d')
msf["date"] = [end_of_month(date) for date in msf["date"]]
msf.columns = [i.lower() for i in msf.columns]
msf["month"] = [date.month for date in msf["date"]]
msf["year"] = msf.date.apply(lambda x:x.year)
# formatting of mse

In [4]:
%%time
mse = pd.read_csv("mse.csv")
mse.columns = [i.lower() for i in mse.columns]
mse["exdt"] = pd.to_datetime(mse["exdt"], format='%Y%m%d')
mse["dclrdt"] = pd.to_datetime(mse["dclrdt"], format='%Y%m%d')
mse["paydt"] = pd.to_datetime(mse["paydt"], format='%Y%m%d')
mse = mse >> mask(X.divamt == X.divamt) ## dropna subset == divamt
mse["date"] = [end_of_month(date) for date in mse["exdt"]]
# merge mse and msf
data = pd.merge(msf, mse, how="outer", on=["permno", "date"])
data["mcap"] = data["prc"] * data["shrout"]
# fill na with distcd if the stock has distcd before so that we can identify stock that pay dividend
# anvd stock that don't pay
data["distcd"] = data.groupby("permno")["distcd"].transform(lambda x: x.fillna(method="ffill"))
# take out the first two digit of the dist code which represent what kind of dividend is pay during the distrubtion
data['freq'] = data.distcd.apply(lambda x:str(x)[:3])

Wall time: 23.1 s


In [5]:
data.head()

,permno,date,shrcd,hexcd,bidlo,askhi,prc,vol,ret,bid,...,retx,month,year,distcd,divamt,dclrdt,exdt,paydt,mcap,freq
0,10000,1985-12-31,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,12,1985,NaN,NaN,NaT,NaT,NaT,NaN,nan
1,10000,1986-01-31,10.0,3,-2.50,-4.4375,-4.3750,1771.0,C,NaN,...,C,1,1986,NaN,NaN,NaT,NaT,NaT,-16100.0,nan
2,10000,1986-02-28,10.0,3,-3.25,-4.3750,-3.2500,828.0,-0.257143,NaN,...,-0.257143,2,1986,NaN,NaN,NaT,NaT,NaT,-11960.0,nan
3,10000,1986-03-31,10.0,3,-3.25,-4.4375,-4.4375,1078.0,0.365385,NaN,...,0.365385,3,1986,NaN,NaN,NaT,NaT,NaT,-16330.0,nan
4,10000,1986-04-30,10.0,3,-4.00,-4.3125,-4.0000,957.0,-0.098592,NaN,...,-0.098592,4,1986,NaN,NaN,NaT,NaT,NaT,-15172.0,nan


In [6]:
data["prc_lag"] = data.groupby("permno")["prc"].shift(1)
# data set with stock that pay div and stock that don't pay (divtype = nan)
data['turnover'] = data['vol']/data['shrout']
data['spread_2'] = data['ask']-data['bid']

In [7]:
data = data >> arrange(X.permno, X.date)

In [8]:
for i in range(12):
    div_lag = 'div_lag' + str(i + 1)
    data[div_lag] = data.groupby('permno')['divamt'].shift(i+1)

In [9]:
## filter on the basic conditions.
dataUse = data.query('date <= "2011-12-31" &'
                          'shrcd in([10,11]) &'
                          'hexcd in([1,2,3]) &'
                          'prc_lag >= 5 &'
                          'ret not in(["B","C"])')

In [10]:
dataUse['ret'] = dataUse.ret.astype('float')

C:\Users\sherrywei\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### <font face = 'Times New Roman'>Panel C: Percent of firm-months with dividend in the last year

In [11]:
DivFreq = pd.DataFrame(dataUse.groupby(['year','freq'])['freq'].count())
DivFreq['percent'] = DivFreq.freq/DivFreq.groupby(level = 0).freq.transform(np.sum)
DivFreq.columns = ['count','percent']
DivFreq = DivFreq.reset_index()

In [12]:
DivFreq.groupby('freq')['percent'].mean()

freq
120    0.000066
121    0.160730
122    0.003501
123    0.562584
124    0.016432
125    0.010084
126    0.004588
127    0.030400
128    0.002425
129    0.002162
131    0.001126
132    0.000367
133    0.002142
134    0.001562
135    0.000482
136    0.000156
137    0.000475
171    0.001155
177    0.000353
181    0.001529
187    0.001321
199    0.000180
nan    0.207178
Name: percent, dtype: float64

<font face = "Times New Roman"> Panel B summary statistics of non-dividend companies

In [13]:
NoDivData = dataUse.query('freq not in(["120","121","123","124","125"])')

In [14]:
NoDivData.shape[0] ## number of firm months

703120

In [15]:
len(set(NoDivData.permno)) ## number of firms

15916

In [16]:
NoDivData[["mcap", "spread_2", "permno","turnover"]].describe().T

,count,mean,std,min,25%,50%,75%,max
mcap,699168.0,904461.272922,6.148122e+06,-1.776986e+07,43704.000000,139517.750000,462638.077500,6.024329e+08
spread_2,560088.0,0.371720,4.307926e+00,-3.312500e+00,0.050000,0.210000,0.375000,6.500000e+02
permno,703120.0,62963.804737,2.730275e+04,1.000100e+04,40125.000000,77089.000000,83800.000000,9.343600e+04
turnover,702615.0,1.535890,2.392034e+00,0.000000e+00,0.358867,0.875369,1.904061,3.041669e+02


### <font face = 'Times New Roman'>  Panel C-Column 2: Percent of dividend observations

In [17]:
divData = dataUse.query('freq in(["120","121","123","124","125"])').dropna(subset = ['distcd'])

In [18]:
DivFreq2 = pd.DataFrame(divData.groupby('freq')['freq'].count())

In [19]:
DivFreq2.columns = ['count']

In [20]:
DivFreq2

,count
freq,
120,5
121,135487
123,1126076
124,37890
125,24153


In [21]:
DivFreq2['percent'] = DivFreq2['count']/DivFreq2['count'].sum()

In [22]:
DivFreq2 ## the second column of panel C

,count,percent
freq,,
120,5,0.000004
121,135487,0.102362
123,1126076,0.850761
124,37890,0.028626
125,24153,0.018248


In [23]:
divData['turnover'] = divData['vol']/divData['shrout']
divData['spread_2'] = divData['ask'] -divData['bid']

In [24]:
divData.sort_values(by = ['permno','date'],inplace = True)

In [25]:
divData['ret'] = divData.ret.astype('float')

In [26]:
table2_mean = {}
table2_std = {}
table2_all_div_prob = {}
table2_quar_div_prob = {}

In [27]:
div_lag_list = divData.columns[-12:].tolist()

In [28]:
for i,div_lag in enumerate(div_lag_list):
    table2_mean[str(i + 1)] = divData.dropna(subset = [div_lag] )['ret'].mean()*100
    table2_std[str(i + 1)] = divData.dropna(subset = [div_lag] )['ret'].std()*100
    table2_all_div_prob[str(i+1)] = len(divData.dropna(subset = [div_lag,'divamt']))/len(divData.dropna(subset = [div_lag]))
    divData_quar = divData.query('freq == "123"')
    table2_quar_div_prob[str(i + 1)] = len(divData_quar.dropna(subset = [div_lag,'divamt']))/len(divData_quar.dropna(subset = [div_lag]))

In [29]:
table2_panelA = pd.DataFrame(pd.Series(table2_mean),columns = ['Mean return'])
table2_panelA['Standard Deviation'] = table2_std.values()
table2_panelA['All dividends'] = table2_all_div_prob.values()
table2_panelA['quar dividends'] = table2_quar_div_prob.values()

In [30]:
table2_panelA ## 

,Mean return,Standard Deviation,All dividends,quar dividends
1,1.064098,9.701810,0.020439,0.018067
2,1.199184,9.814587,0.054591,0.054148
3,1.370024,9.585290,0.843451,0.868933
4,1.022117,9.697631,0.095163,0.092383
5,1.190069,9.842323,0.069989,0.067457
6,1.383106,9.633087,0.816145,0.831705
7,1.046374,9.739536,0.107245,0.101101
8,1.163598,9.867143,0.078547,0.076900
9,1.360605,9.622304,0.793025,0.820768
10,1.016591,9.776346,0.108431,0.103511


In [ ]:
## 这个table 看上去还行，但是1，4，7，8的prob好像有点偏高。

In [31]:
for col in ['bidlo','askhi']:
    divData.pop(col)

In [32]:
divData.columns

Index(['permno', 'date', 'shrcd', 'hexcd', 'prc', 'vol', 'ret', 'bid', 'ask',
       'shrout', 'spread', 'retx', 'month', 'year', 'distcd', 'divamt',
       'dclrdt', 'exdt', 'paydt', 'mcap', 'freq', 'prc_lag', 'turnover',
       'spread_2', 'div_lag1', 'div_lag2', 'div_lag3', 'div_lag4', 'div_lag5',
       'div_lag6', 'div_lag7', 'div_lag8', 'div_lag9', 'div_lag10',
       'div_lag11', 'div_lag12'],
      dtype='object')

In [33]:
divData[['date','permno','freq','divamt','div_lag1','div_lag2','div_lag3','div_lag4']] >> head()

,date,permno,freq,divamt,div_lag1,div_lag2,div_lag3,div_lag4
29,1986-10-31,10001,123,NaN,0.105,NaN,NaN,0.105
30,1986-11-30,10001,123,NaN,NaN,0.105,NaN,NaN
31,1986-12-31,10001,123,0.105,NaN,NaN,0.105,NaN
32,1987-01-31,10001,123,NaN,0.105,NaN,NaN,0.105
33,1987-02-28,10001,123,NaN,NaN,0.105,NaN,NaN


In [34]:
divData_pred = divData.query('(freq == "123" & div_lag3 == div_lag3)|'
                             '(freq == "124" & div_lag6 == div_lag6)|'
                             '(freq == "125" & div_lag12 == div_lag12)')

In [35]:
probs = [0.01,0.05,0.25,0.5,0.75,0.95,0.99]

In [36]:
def table2PanelBpart(df):
    table2PanelBpart = pd.DataFrame(np.array([df.ret.quantile(prob) for prob in probs]).reshape(1,7),columns = probs)
    table2PanelBpart.insert(loc = 0,column = 'MeanReturn',value = df['ret'].mean())
    table2PanelBpart.insert(loc = 1,column = 'StandardDeviation',value = df['ret'].std())
    table2PanelBpart >> mutate(sharp_ratio = X.MeanReturn/X.StandardDeviation)
    return table2PanelBpart

In [37]:
div_No_pred_list = set(divData.index.tolist())-set(divData_pred.index.tolist())
divData_notPred = divData.loc[div_No_pred_list,]

In [38]:
table2PanelB = pd.concat([table2PanelBpart(df) for df in [divData_pred,divData_notPred,NoDivData]])

In [39]:
table2PanelB

,MeanReturn,StandardDeviation,0.01,0.05,0.25,0.5,0.75,0.95,0.99
0,0.013994,0.096203,-0.226792,-0.127829,-0.037575,0.009524,0.060800,0.168602,0.294905
0,0.011186,0.108734,-0.261213,-0.147006,-0.045776,0.005464,0.061856,0.185714,0.335892
0,0.007335,0.169407,-0.404349,-0.239316,-0.080808,0.000000,0.084507,0.274390,0.509434


### <font face = 'Times New Roman'>table 1 merge with gvkey

In [40]:
import feather

In [41]:
linkTable = feather.read_dataframe('link_ccmxpf_linktable.feather')

In [42]:
linkTable.head()

,gvkey,linkprim,liid,linktype,lpermno,lpermco,USEDFLAG,linkdt,linkenddt
0,001000,C,00X,NU,NaN,NaN,-1,1961-01-01,1970-09-29
1,001000,P,01,NU,NaN,NaN,-1,1970-09-30,1970-11-12
2,001000,P,01,LU,25881.0,23369.0,1,1970-11-13,1978-06-30
3,001001,C,00X,NU,NaN,NaN,-1,1978-01-01,1983-09-19
4,001001,P,01,LU,10015.0,6398.0,1,1983-09-20,1986-07-31


In [67]:
atq= pd.read_csv('atq.csv')

In [72]:
atq.datadate = pd.to_datetime(atq.datadate,format = '%Y%m%d')

In [73]:
atq.rename(columns = {'GVKEY':'gvkey'},inplace = True)

In [74]:
linkTable['gvkey'] = linkTable.gvkey.astype('int')

In [75]:
linkTable.dropna(subset = ['lpermno'],inplace = True)

In [76]:
atq['gvkey'] = atq.gvkey.astype('str')

In [50]:
linkTable['gvkey'] = linkTable.gvkey.astype('str')

In [51]:
linkTable['gvkey'] = linkTable.gvkey.astype('str')

In [109]:
linkTable.query('count == 2').head()

,gvkey,linkprim,liid,linktype,lpermno,lpermco,USEDFLAG,linkdt,linkenddt,count
17,1007,C,00X,LU,10058.0,20.0,1,1973-10-01,1979-01-30,2
18,1007,P,01,LU,10058.0,20.0,1,1979-01-31,1984-09-28,2
26,1010,C,00X,LU,10006.0,22156.0,1,1950-05-01,1962-01-30,2
27,1010,P,01,LU,10006.0,22156.0,1,1962-01-31,1984-06-28,2
46,1018,C,00X,LU,10162.0,88.0,1,1973-07-01,1979-01-30,2


In [112]:
linkTable['count'] = linkTable.groupby('gvkey')['gvkey'].transform('count')

In [111]:
linkTable.drop_duplicates(subset = ['gvkey','lpermno'],keep = 'first',inplace = True)

In [118]:
divData.query('permno == 80071.0')

,permno,date,shrcd,hexcd,prc,vol,ret,bid,ask,shrout,...,div_lag3,div_lag4,div_lag5,div_lag6,div_lag7,div_lag8,div_lag9,div_lag10,div_lag11,div_lag12


In [119]:
linkTable.query('count == 2').head()

,gvkey,linkprim,liid,linktype,lpermno,lpermco,USEDFLAG,linkdt,linkenddt,count
106,1043,C,00X,LN,18980.0,20009.0,-1,1949-08-01,1950-12-29,2
112,1043,C,01,LC,80071.0,30061.0,1,1993-12-06,2000-01-27,2
172,1072,C,00X,LN,56899.0,25544.0,-1,1973-01-01,1973-03-29,2
175,1072,C,01,LC,81912.0,30930.0,1,1995-08-15,9999-01-01,2
185,1076,P,01,LC,10517.0,5674.0,1,1982-11-04,1992-12-31,2


In [52]:
atq.dtypes

gvkey        object
datadate      int64
fyearq        int64
fqtr          int64
indfmt       object
consol       object
popsrc       object
datafmt      object
conm         object
curcdq       object
datacqtr     object
datafqtr     object
atq         float64
saleq       float64
costat       object
dtype: object

In [79]:
linkTable['gvkey'] = linkTable.gvkey.astype('str')

In [86]:
atq[['gvkey','datadate','atq']].shape

(1120835, 3)

In [120]:
atq_permno = atq[['gvkey','datadate','atq']].merge(linkTable[['gvkey','lpermno']],on = 'gvkey',how = 'left')

In [82]:
linkTable.query('lpermno == 10001')

,gvkey,linkprim,liid,linktype,lpermno,lpermco,USEDFLAG,linkdt,linkenddt
28929,12994,P,01,LC,10001.0,7953.0,1,1986-01-09,9999-01-01


In [98]:
atq_permno.dropna(subset = ['atq'],inplace = True)

In [99]:
atq_permno.groupby('gvkey')['gvkey'].count()

gvkey
1000       14
10000     177
100004     17
10001     106
100011      6
10002     342
100039     28
10004     102
100049      5
10005     162
10006     156
100075     41
10008     179
100080     21
10009      12
100095     16
1001       11
10010      64
10011      12
100114     20
10012      26
10013      78
10014      26
10015       8
10016     180
100165     28
10017      22
100172     14
10020      54
10021      76
         ... 
9953        8
9954      124
9955       24
9956        8
9958       27
9959       25
9962        8
9963      110
9964       31
9965      140
9966       58
9967      177
9968       29
9969      268
9971       46
9973       40
9975       12
9977       57
9981       51
9984       28
9985      246
9986       24
9988       92
9989       21
9990       31
9993       22
9994       30
9997       48
9998      172
9999      122
Name: gvkey, Length: 23129, dtype: int64

In [121]:
atq_permno.shape

(1218295, 4)

In [81]:
atq_permno.sort_values(['gvkey','datadate']).tail()

,gvkey,datadate,atq,lpermno
641705,9999,2011-05-31,560.591,72486.0
641706,9999,2011-08-31,552.848,72486.0
641707,9999,2011-11-30,523.497,72486.0
641708,9999,2012-02-29,513.986,72486.0
641709,9999,2012-05-31,531.462,72486.0


In [122]:
atq_permno.dropna(subset = ['lpermno'],inplace = True)
atq_permno.dropna(subset = ['atq'],inplace = True)

In [124]:
atq_permno.dtypes

gvkey               object
datadate    datetime64[ns]
atq                float64
lpermno            float64
dtype: object

In [128]:
atq_permno.rename(columns = {'lpermno':'permno'},inplace = True)

In [129]:
atq_permno.dtypes

gvkey               object
datadate    datetime64[ns]
atq                float64
permno             float64
dtype: object

In [130]:
atq_permno.rename(columns = {'datadate':'date'},inplace = True)

In [131]:
divDataAtq= pd.merge(divData,atq_permno[['permno','date','atq']],on = ['permno','date'],how = 'left')

In [139]:
divDataBM = divDataAtq.query('atq == atq & mcap == mcap') >> mutate(BM = X.atq/X.mcap)

In [143]:
divDataBM['BM'].describe()

count    281037.000000
mean          0.003629
std           0.010291
min          -0.970627
25%           0.000935
50%           0.001867
75%           0.003987
max           1.291032
Name: BM, dtype: float64

In [ ]:
divDataAtq[['permno','date','mcap','atq']]

In [ ]:
divDataBM = divDataAtq.dropna(subset = ['mcap','atq'])>>mutate(BM = (X.atq)/X.mcap)

In [ ]:
divDataBM[['BM']].describe().T

In [ ]:
fundamental.head()

In [ ]:
divDataBM.BM

In [ ]:
fundamental_permno.query('gvkey == 1000')

In [ ]:
1359690/4